In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
%matplotlib inline
import seaborn as sns
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix,classification_report
# np.set_printoptions(threshold=np.nan)

# Below XGBoosts models are having best of all AUC Metric and also detecting best amount of defaulters as compared to other Models

In [2]:
df_train = pd.read_csv("FINALHOMECREDIT.csv")


In [3]:
df_train.head(5)

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,CC_NAME_CONTRACT_STATUS_Signed_MAX,CC_NAME_CONTRACT_STATUS_Signed_MEAN,CC_NAME_CONTRACT_STATUS_Signed_SUM,CC_NAME_CONTRACT_STATUS_Signed_VAR,CC_NAME_CONTRACT_STATUS_nan_MIN,CC_NAME_CONTRACT_STATUS_nan_MAX,CC_NAME_CONTRACT_STATUS_nan_MEAN,CC_NAME_CONTRACT_STATUS_nan_SUM,CC_NAME_CONTRACT_STATUS_nan_VAR,CC_COUNT
0,100002,1,0,0,0,0,202500.0,406597.5,24700.5,351000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100003,0,1,0,1,0,270000.0,1293502.5,35698.5,1129500.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100004,0,0,1,0,0,67500.0,135000.0,6750.0,135000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100006,0,1,0,0,0,135000.0,312682.5,29686.5,297000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0
4,100007,0,0,0,0,0,121500.0,513000.0,21865.5,513000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df_train.shape

(307507, 798)

In [9]:
df_train["TARGET"].nunique()

2

In [10]:
df_train["TARGET"].value_counts(dropna=False)

0    282682
1     24825
Name: TARGET, dtype: int64

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( df_train.drop('TARGET',axis=1) , df_train['TARGET'] , test_size=0.2, random_state=42)
model = XGBClassifier(max_depth=6, learning_rate=0.1,n_estimators=100,n_jobs=16,scale_pos_weight=4,missing=np.nan,gamma=16, eval_metric="auc",reg_lambda=40,reg_alpha=40)
model.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, eval_metric='auc', gamma=16, learning_rate=0.1,
       max_delta_step=0, max_depth=6, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=16, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=40,
       reg_lambda=40, scale_pos_weight=4, seed=None, silent=True,
       subsample=1)

In [12]:
from sklearn.metrics import roc_auc_score
y_train_predicted = model.predict_proba(X_train)[:,1]
y_test_predicted = model.predict_proba(X_test)[:,1]
print("Train AUC %.4f"%roc_auc_score(y_train,y_train_predicted))
print("Test AUC %.4f"%roc_auc_score(y_test,y_test_predicted))

Train AUC 0.8229
Test AUC 0.7818


In [13]:
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)
print("Train Accuracy %.4f" % accuracy_score(y_train,y_train_pred))
print("Test Accuracy %.4f" % accuracy_score(y_test,y_test_pred))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Train Accuracy 0.8980
Test Accuracy 0.8917


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [15]:
from sklearn.metrics import confusion_matrix

confmat=confusion_matrix(y_test,y_test_pred)
print(confmat)
# plt.imshow(confmat, cmap='binary')

[[53235  3246]
 [ 3417  1604]]


In [16]:

# Function to calculate missing values by column# Funct 
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [17]:
# Missing values statistics
missing_values = missing_values_table(df_train)
missing_values.index[:45]

Your selected dataframe has 798 columns.
There are 604 columns that have missing values.


Index(['REFUSED_RATE_DOWN_PAYMENT_MIN', 'REFUSED_RATE_DOWN_PAYMENT_MEAN',
       'REFUSED_RATE_DOWN_PAYMENT_MAX', 'REFUSED_AMT_DOWN_PAYMENT_MEAN',
       'REFUSED_AMT_DOWN_PAYMENT_MAX', 'REFUSED_AMT_DOWN_PAYMENT_MIN',
       'REFUSED_APP_CREDIT_PERC_VAR', 'CC_AMT_PAYMENT_CURRENT_VAR',
       'CC_CNT_DRAWINGS_OTHER_CURRENT_VAR', 'CC_AMT_DRAWINGS_ATM_CURRENT_VAR',
       'CC_CNT_DRAWINGS_ATM_CURRENT_VAR', 'CC_CNT_DRAWINGS_POS_CURRENT_VAR',
       'CC_AMT_DRAWINGS_OTHER_CURRENT_VAR', 'CC_AMT_DRAWINGS_POS_CURRENT_VAR',
       'CC_AMT_PAYMENT_CURRENT_MEAN', 'CC_AMT_PAYMENT_CURRENT_MAX',
       'CC_AMT_PAYMENT_CURRENT_MIN', 'CC_CNT_DRAWINGS_ATM_CURRENT_MEAN',
       'CC_AMT_DRAWINGS_OTHER_CURRENT_MEAN',
       'CC_AMT_DRAWINGS_OTHER_CURRENT_MAX', 'CC_CNT_DRAWINGS_ATM_CURRENT_MAX',
       'CC_CNT_DRAWINGS_ATM_CURRENT_MIN', 'CC_AMT_DRAWINGS_POS_CURRENT_MIN',
       'CC_AMT_DRAWINGS_POS_CURRENT_MAX', 'CC_AMT_DRAWINGS_OTHER_CURRENT_MIN',
       'CC_CNT_DRAWINGS_OTHER_CURRENT_MIN',
       'CC_CNT

In [18]:
from sklearn.model_selection import train_test_split
newX=df_train.drop(missing_values.index[:15],axis=1)
X_train, X_test, y_train, y_test = train_test_split( newX.drop('TARGET',axis=1) , df_train['TARGET'] , test_size=0.2, random_state=42)
model = XGBClassifier(max_depth=6, learning_rate=0.1,n_estimators=100,n_jobs=16,scale_pos_weight=4,missing=np.nan,gamma=16, eval_metric="auc",reg_lambda=40,reg_alpha=40)
model.fit(X_train,y_train)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, eval_metric='auc', gamma=16, learning_rate=0.1,
       max_delta_step=0, max_depth=6, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=16, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=40,
       reg_lambda=40, scale_pos_weight=4, seed=None, silent=True,
       subsample=1)

In [19]:
# df_train.drop(missing_values.index[:30],axis=1)

In [20]:
from sklearn.metrics import roc_auc_score
y_train_predicted = model.predict_proba(X_train)[:,1]
y_test_predicted = model.predict_proba(X_test)[:,1]
print("Train AUC %.4f"%roc_auc_score(y_train,y_train_predicted))
print("Test AUC %.4f"%roc_auc_score(y_test,y_test_predicted))

Train AUC 0.8229
Test AUC 0.7813


In [21]:
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)
print("Train Accuracy %.4f" % accuracy_score(y_train,y_train_pred))
print("Test Accuracy %.4f" % accuracy_score(y_test,y_test_pred))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Train Accuracy 0.8984
Test Accuracy 0.8927


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [22]:
from sklearn.metrics import confusion_matrix

confmat=confusion_matrix(y_test,y_test_pred)
print(confmat)
# plt.imshow(confmat, cmap='binary')

[[53303  3178]
 [ 3419  1602]]
